In [1]:
import json
import os
import shutil
import subprocess
import time

import numpy as np
import pandas as pd

In [2]:
def run_WBM(
    sim_ID,
    run_start,
    run_end,
    spinup_start,
    spinup_end,
    output_vars,
    network,
    pet_method,
    airT_primary,
    airT_secondary,
    airT_min,
    airT_max,
    precip_primary,
    precip_secondary,
    cloudFr_primary,
    windU_primary,
    windV_primary,
    humidity_primary,
    albedo_primary,
    lai,
    land_collapse,
    crop_area_frac,
    crop_area_frac_patch,
    crop_par,
    crop_par_patch,
    main_path="./",
    project_ID="wbm_test_runs",
    spinup_loops=5,
):
    # Returns a blank string if parameter is empty
    def data_init_path(par):
        if par == "":
            return ""
        else:
            return sim_path + "/data_init/" + par

    #################
    # Set up folders
    #################
    sim_path = main_path + sim_ID

    if os.path.isdir(sim_path):
        print("Directory already exists!")
    else:
        shutil.copytree(main_path + "wbm_storage_template", sim_path)

    ###################
    # Update init file
    ###################
    with open("./wbm_init_template.json") as f:
        wbm_init = json.load(f)

    # These are typically the settings we care about but there are more!
    # Organization
    wbm_init["ID"] = sim_ID
    wbm_init["Comment"] = ""
    wbm_init["Project"] = project_ID

    wbm_init["MT_Code_Name"]["MT_ID"] = sim_ID
    wbm_init["MT_Code_Name"]["Output_dir"] = sim_path + "/wbm_output"
    wbm_init["MT_Code_Name"]["Run_Start"] = run_start
    wbm_init["MT_Code_Name"]["Run_End"] = run_end

    wbm_init["Spinup"]["Start"] = spinup_start
    wbm_init["Spinup"]["End"] = spinup_end
    wbm_init["Spinup"]["Loops"] = spinup_loops

    wbm_init["Network"] = network
    
    # Output variables
    wbm_init['Output_vars'] =  '\n '.join(output_vars)

    # Params
    wbm_init["PET"] = pet_method

    # Climate
    wbm_init["MT_airT"]["Primary"] = data_init_path(airT_primary)
    wbm_init["MT_airT"]["Secondary"] = data_init_path(airT_secondary)
    wbm_init["MT_airT"]["airTmin"] = data_init_path(airT_min)
    wbm_init["MT_airT"]["airTmax"] = data_init_path(airT_max)

    wbm_init["MT_Precip"]["Primary"] = sim_path + "/data_init/" + precip_primary
    wbm_init["MT_Precip"]["Secondary"] = data_init_path(precip_secondary)

    wbm_init["MT_windU"]["Primary"] = data_init_path(windU_primary)
    wbm_init["MT_windV"]["Primary"] = data_init_path(windV_primary)
    wbm_init["MT_humidity"]["Primary"] = data_init_path(humidity_primary)
    wbm_init["MT_albedo"]["Primary"] = data_init_path(albedo_primary)

    wbm_init["MT_cloudFr"]["Primary"] = sim_path + "/data_init/" + cloudFr_primary

    # Crop
    wbm_init["MT_LAI"] = data_init_path(lai)

    wbm_init["landCollapse"] = land_collapse

    wbm_init["Irrigation"]["CropAreaFrac"] = sim_path + "/data_init/" + crop_area_frac
    wbm_init["Irrigation"]["CropAreaFracPatch"] = data_init_path(crop_area_frac_patch)

    wbm_init["Irrigation"]["CropParFile"] = sim_path + "/data/" + crop_par
    wbm_init["Irrigation"]["CropParFilePatch"] = data_init_path(crop_par_patch)

    # Save init file
    f = open(sim_path + "/wbm_init/wbm.init", "w")
    f.write("{ \n")
    for key in wbm_init:
        # Subkeys
        if type(wbm_init[key]) == dict:
            f.write(key + " => {\n")
            for subkey in wbm_init[key]:
                # Skip writing if entry = XXX
                if str(wbm_init[key][subkey])[-3:] == "XXX":
                    continue
                f.write(subkey + " => '" + str(wbm_init[key][subkey]) + "',\n")
            f.write("},\n")
        # Regalar keys
        else:
            # Skip writing if entry = XXX
            if str(wbm_init[key])[-3:] == "XXX":
                continue
            f.write(key + " => '" + str(wbm_init[key]) + "',\n")
    f.write("} \n")
    f.close()
    
    ####################
    # Update crop files
    ####################
    df_crop = pd.read_csv(main_path + sim_ID + "/data/" + crop_par)
    df_crop = df_crop.applymap(lambda x: x.replace("PATH_HERE", main_path + sim_ID) if type(x)==str else x)
    df_crop.to_csv(main_path + sim_ID + "/data/" + crop_par, index=False)

    ################
    # Run jobs
    ################
    ##### -noRun
    args = "sim_dir=" + sim_ID
    out = sim_path + "/check_run_WBM.out"

    if os.path.isfile(sim_path + "/check_run_WBM.out"):
        check_job_out = subprocess.run(
            ["tail", "-n", "5", sim_path + "/check_run_WBM.out"],
            capture_output=True,
            text=True,
        ).stdout.split("\n")[-4]
        if check_job_out == "All Done!":
            print("Check already completed.")
        else:
            os.remove(sim_path + "/check_run_WBM.out")

            check_jobid = subprocess.run(
                ["qsub", "-v", args, "-o", out, sim_path + "/check_run_WBM.pbs"],
                capture_output=True,
                text=True,
            ).stdout.split(".")[0]
            print("check jobid: " + check_jobid)
    else:
        check_jobid = subprocess.run(
            ["qsub", "-v", args, "-o", out, sim_path + "/check_run_WBM.pbs"],
            capture_output=True,
            text=True,
        ).stdout.split(".")[0]
        print("check jobid: " + check_jobid)

    # check for errors in setup
    while not os.path.isfile(sim_path + "/check_run_WBM.out"):
        time.sleep(60)
    check_job_out = subprocess.run(
        ["tail", "-n", "5", sim_path + "/check_run_WBM.out"],
        capture_output=True,
        text=True,
    ).stdout.split("\n")[-4]
    if check_job_out != "All Done!":
        print("Something went wrong!")
        return None

    ####### Spool and run
    args = "sim_dir=" + sim_ID
    out = sim_path + "/spool_WBM.out"

    # check if spooling already done
    spooling_complete = False
    if os.path.isfile(sim_path + "/spool_WBM.out"):
        check_job_out = subprocess.run(
            ["tail", "-n", "5", sim_path + "/spool_WBM.out"],
            capture_output=True,
            text=True,
        ).stdout.split("\n")[-4]
        if check_job_out == "All Done!":
            print("Spooling already completed.")
            spooling_complete = True
        else:
            os.remove(sim_path + "/spool_WBM.out")
            spool_jobid = subprocess.run(
                ["qsub", "-v", args, "-o", out, sim_path + "/spool_WBM.pbs"],
                capture_output=True,
                text=True,
            ).stdout.split(".")[0]
            print("spool jobid: " + spool_jobid)
    else:
        spool_jobid = subprocess.run(
            ["qsub", "-v", args, "-o", out, sim_path + "/spool_WBM.pbs"],
            capture_output=True,
            text=True,
        ).stdout.split(".")[0]
        print("spool jobid: " + spool_jobid)

    ########## Run WBM (when spooling is complete)
    args = "sim_dir=" + sim_ID
    out = sim_path + "/run_WBM.out"

    if spooling_complete:
        run_jobid = subprocess.run(
            ["qsub", "-v", args, "-o", out, sim_path + "/run_WBM.pbs"],
            capture_output=True,
            text=True,
        ).stdout.split(".")[0]
        print("run jobid: " + run_jobid)
    else:
        run_jobid = subprocess.run(
            [
                "qsub",
                "-W",
                "depend=afterok:" + spool_jobid,
                "-v",
                args,
                "-o",
                out,
                sim_path + "/run_WBM.pbs",
            ],
            capture_output=True,
            text=True,
        ).stdout.split(".")[0]
        print("run jobid: " + run_jobid)

# Test running WBM simulations

## PRISM + MERRA 

In [4]:
# simulation 0: hindcast, Hamon PET, Temp + Precip only, crop averages
run_WBM(
    sim_ID="PRISM-test_soilM_HamonTP_cropAverage",
    run_start="1981-01-01",
    run_end="1985-12-31",
    spinup_start="1981-01-01",
    spinup_end="1981-12-31",
    output_vars=['soilMoist', 'soilMoistFrac', 'airT', 'precip'],
    network="/gpfs/group/kaf26/default/private/WBM_data/squam.sr.unh.edu/US_CDL_v3_data/network/flowdirection206_us.asc",
    pet_method="Hamon",
    airT_primary="PRISM_tmean_4kmD2_d.init",
    airT_secondary="",
    airT_min="",
    airT_max="",
    precip_primary="PRISM_ppt_4kmD2_d.init",
    precip_secondary="",
    windU_primary="",
    windV_primary="",
    humidity_primary="",
    albedo_primary="",
    cloudFr_primary="merra2_cldtot_d.init",
    lai="merra2_lai2_dc.init",
    land_collapse="average",
    crop_area_frac="cdl_cropland.init",
    crop_area_frac_patch="",
    crop_par="CDL-US-M_CropParameters.csv",
    crop_par_patch="",
    main_path="/gpfs/group/kaf26/default/dcl5300/test-runs_WBM/current/",
    project_ID="PCHES_crop_yield_UC_climate_param",
    spinup_loops=5,
)

Directory already exists!
Check already completed.
spool jobid: 43697544
run jobid: 43697545


In [3]:
# simulation 1: hindcast, Hamon PET, Temp + Precip only, crop averages
run_WBM(
    sim_ID="PRISM_soilM_HamonTP_cropAverage",
    run_start="1981-01-01",
    run_end="2015-12-31",
    spinup_start="1981-01-01",
    spinup_end="1981-12-31",
    output_vars=['soilMoist', 'soilMoistFrac'],
    network="/gpfs/group/kaf26/default/private/WBM_data/squam.sr.unh.edu/US_CDL_v3_data/network/flowdirection206_us.asc",
    pet_method="Hamon",
    airT_primary="PRISM_tmean_4kmD2_d.init",
    airT_secondary="",
    airT_min="",
    airT_max="",
    precip_primary="PRISM_ppt_4kmD2_d.init",
    precip_secondary="",
    windU_primary="",
    windV_primary="",
    humidity_primary="",
    albedo_primary="",
    cloudFr_primary="merra2_cldtot_d.init",
    lai="merra2_lai2_dc.init",
    land_collapse="average",
    crop_area_frac="cdl_cropland.init",
    crop_area_frac_patch="",
    crop_par="CDL-US-M_CropParameters.csv",
    crop_par_patch="",
    main_path="/gpfs/group/kaf26/default/dcl5300/test-runs_WBM/current/",
    project_ID="PCHES_crop_yield_UC_climate_param",
    spinup_loops=5,
)

Directory already exists!
Check already completed.
spool jobid: 43714539
run jobid: 43714540


In [3]:
# simulation 2: hindcast, Hamon PET, Temp + Precip only, individual crops
run_WBM(
    sim_ID="PRISM_soilM_HamonTP_cropIndividual",
    run_start="1981-01-01",
    run_end="2015-12-31",
    spinup_start="1981-01-01",
    spinup_end="1981-12-31",
    output_vars=['soilMoist', 'soilMoistFrac', 'airT', 'precip'],
    network="/gpfs/group/kaf26/default/private/WBM_data/squam.sr.unh.edu/US_CDL_v3_data/network/flowdirection206_us.asc",
    pet_method="Hamon",
    airT_primary="PRISM_tmean_4kmD2_d.init",
    airT_secondary="",
    airT_min="",
    airT_max="",
    precip_primary="PRISM_ppt_4kmD2_d.init",
    precip_secondary="",
    windU_primary="",
    windV_primary="",
    humidity_primary="",
    albedo_primary="",
    cloudFr_primary="merra2_cldtot_d.init",
    lai="merra2_lai2_dc.init",
    land_collapse="",
    crop_area_frac="cdl_cropland.init",
    crop_area_frac_patch="",
    crop_par="CDL-US_CropParameters.csv",
    crop_par_patch="",
    main_path="/gpfs/group/kaf26/default/dcl5300/test-runs_WBM/current/",
    project_ID="PCHES_crop_yield_UC_climate_param",
    spinup_loops=5,
)

check jobid: 43726745
spool jobid: 43726946
run jobid: 43726947


In [5]:
# simulation 2: hindcast, Hamon PET, Temp + Precip only, individual crops
run_WBM(
    sim_ID="test-PRISM_soilM_HamonTP_cropIndividual",
    run_start="1981-01-01",
    run_end="1983-12-31",
    spinup_start="1981-01-01",
    spinup_end="1981-12-31",
    output_vars=['soilMoist', 'soilMoistFrac'],
    network="/gpfs/group/kaf26/default/private/WBM_data/squam.sr.unh.edu/US_CDL_v3_data/network/flowdirection206_us.asc",
    pet_method="Hamon",
    airT_primary="PRISM_tmean_4kmD2_d.init",
    airT_secondary="",
    airT_min="",
    airT_max="",
    precip_primary="PRISM_ppt_4kmD2_d.init",
    precip_secondary="",
    windU_primary="",
    windV_primary="",
    humidity_primary="",
    albedo_primary="",
    cloudFr_primary="merra2_cldtot_d.init",
    lai="merra2_lai2_dc.init",
    land_collapse="",
    crop_area_frac="cdl_cropland.init",
    crop_area_frac_patch="",
    crop_par="CDL-US_CropParameters.csv",
    crop_par_patch="",
    main_path="/gpfs/group/kaf26/default/dcl5300/test-runs_WBM/current/",
    project_ID="PCHES_crop_yield_UC_climate_param",
    spinup_loops=5,
)

Directory already exists!
Check already completed.
spool jobid: 43733362
run jobid: 43733363


In [8]:
# simulation 3: hindcast, Penman–Monteith PET (all vars), crop averages
run_WBM(
    sim_ID="PRISM-MERRA_soilM_PM_cropAverage",
    run_start="1981-01-01",
    run_end="2015-12-31",
    spinup_start="1981-01-01",
    spinup_end="1981-12-31",
    output_vars=['soilMoist', 'soilMoistFrac'],
    network="/gpfs/group/kaf26/default/private/WBM_data/squam.sr.unh.edu/US_CDL_v3_data/network/flowdirection206_us.asc",
    pet_method="Penman-Monteith",
    airT_primary="PRISM_tmean_4kmD2_d.init",
    airT_secondary="",
    airT_min="",
    airT_max="",
    precip_primary="PRISM_ppt_4kmD2_d.init",
    precip_secondary="",
    windU_primary="merra2_u10m_d.init",
    windV_primary="merra2_v10m_d.init",
    humidity_primary="merra2_rh2m_d.init",
    albedo_primary="merra_albedo_dc.init",
    cloudFr_primary="merra2_cldtot_d.init",
    lai="merra2_lai2_dc.init",
    land_collapse="average",
    crop_area_frac="cdl_cropland.init",
    crop_area_frac_patch="",
    crop_par="CDL-US-M_CropParameters.csv",
    crop_par_patch="",
    main_path="/gpfs/group/kaf26/default/dcl5300/test-runs_WBM/current/",
    project_ID="PCHES_crop_yield_UC_climate_param",
    spinup_loops=5,
)

check jobid: 43694198
spool jobid: 43694201
run jobid: 43694202


In [9]:
# simulation 4: hindcast, Penman–Monteith PET (all vars), individual crop
run_WBM(
    sim_ID="PRISM-MERRA_soilM_PM_cropIndividual",
    run_start="1981-01-01",
    run_end="2015-12-31",
    spinup_start="1981-01-01",
    spinup_end="1981-12-31",
    output_vars=['soilMoist', 'soilMoistFrac'],
    network="/gpfs/group/kaf26/default/private/WBM_data/squam.sr.unh.edu/US_CDL_v3_data/network/flowdirection206_us.asc",
    pet_method="Penman-Monteith",
    airT_primary="PRISM_tmean_4kmD2_d.init",
    airT_secondary="",
    airT_min="",
    airT_max="",
    precip_primary="PRISM_ppt_4kmD2_d.init",
    precip_secondary="",
    windU_primary="merra2_u10m_d.init",
    windV_primary="merra2_v10m_d.init",
    humidity_primary="merra2_rh2m_d.init",
    albedo_primary="merra_albedo_dc.init",
    cloudFr_primary="merra2_cldtot_d.init",
    lai="merra2_lai2_dc.init",
    land_collapse="",
    crop_area_frac="cdl_cropland.init",
    crop_area_frac_patch="",
    crop_par="CDL-US_CropParameters.csv",
    crop_par_patch="",
    main_path="/gpfs/group/kaf26/default/dcl5300/test-runs_WBM/current/",
    project_ID="PCHES_crop_yield_UC_climate_param",
    spinup_loops=5,
)

check jobid: 43694204
spool jobid: 43694247
run jobid: 43694248


## GMFD 

In [4]:
# simulation 1: hindcast, Hamon PET, Temp + Precip only, crop averages
run_WBM(
    sim_ID="GMFD_soilM_HamonTP_cropAverage",
    run_start="1981-01-01",
    run_end="2015-12-31",
    spinup_start="1981-01-01",
    spinup_end="1981-12-31",
    output_vars=['soilMoist', 'soilMoistFrac'],
    network="/gpfs/group/kaf26/default/private/WBM_data/squam.sr.unh.edu/US_CDL_v3_data/network/flowdirection206_us.asc",
    pet_method="Hamon",
    airT_primary="GMFD_tas_daily.init",
    airT_secondary="",
    airT_min="",
    airT_max="",
    precip_primary="GMFD_prcp_daily.init",
    precip_secondary="",
    windU_primary="",
    windV_primary="",
    humidity_primary="",
    albedo_primary="",
    cloudFr_primary="merra2_cldtot_d.init",
    lai="merra2_lai2_dc.init",
    land_collapse="average",
    crop_area_frac="cdl_cropland.init",
    crop_area_frac_patch="",
    crop_par="CDL-US-M_CropParameters.csv",
    crop_par_patch="",
    main_path="/gpfs/group/kaf26/default/dcl5300/test-runs_WBM/current/",
    project_ID="PCHES_crop_yield_UC_climate_param",
    spinup_loops=5,
)

Directory already exists!
Check already completed.
spool jobid: 43714543
run jobid: 43714544


## Livneh 

In [3]:
# simulation 1: hindcast, Hamon PET, Temp + Precip only, crop averages
run_WBM(
    sim_ID="Livneh_soilM_HamonTP_cropAverage",
    run_start="1981-01-01",
    run_end="2014-12-31",
    spinup_start="1981-01-01",
    spinup_end="1981-12-31",
    output_vars=['soilMoist', 'soilMoistFrac'],
    network="/gpfs/group/kaf26/default/private/WBM_data/squam.sr.unh.edu/US_CDL_v3_data/network/flowdirection206_us.asc",
    pet_method="Hamon",
    airT_primary="XXX",
    airT_secondary="XXX",
    airT_min="Livneh2_Tmin_daily.init",
    airT_max="Livneh2_Tmax_daily.init",
    precip_primary="Livneh2_precip_daily.init",
    precip_secondary="",
    windU_primary="",
    windV_primary="",
    humidity_primary="",
    albedo_primary="",
    cloudFr_primary="merra2_cldtot_d.init",
    lai="merra2_lai2_dc.init",
    land_collapse="average",
    crop_area_frac="cdl_cropland.init",
    crop_area_frac_patch="",
    crop_par="CDL-US-M_CropParameters.csv",
    crop_par_patch="",
    main_path="/gpfs/group/kaf26/default/dcl5300/test-runs_WBM/current/",
    project_ID="PCHES_crop_yield_UC_climate_param",
    spinup_loops=5,
)

Directory already exists!
Check already completed.
spool jobid: 43735625
run jobid: 43735626


## gridMET

In [3]:
# simulation 1: hindcast, Hamon PET, Temp + Precip only, crop averages
run_WBM(
    sim_ID="gridMET_soilM_HamonTP_cropAverage",
    run_start="1981-01-01",
    run_end="2014-12-31",
    spinup_start="1981-01-01",
    spinup_end="1981-12-31",
    output_vars=['soilMoist', 'soilMoistFrac'],
    network="/gpfs/group/kaf26/default/private/WBM_data/squam.sr.unh.edu/US_CDL_v3_data/network/flowdirection206_us.asc",
    pet_method="Hamon",
    airT_primary="XXX",
    airT_secondary="XXX",
    airT_min="gridMET_tmmn_daily.init",
    airT_max="gridMET_tmmx_daily.init",
    precip_primary="gridMET_pr_daily.init",
    precip_secondary="",
    windU_primary="",
    windV_primary="",
    humidity_primary="",
    albedo_primary="",
    cloudFr_primary="merra2_cldtot_d.init",
    lai="merra2_lai2_dc.init",
    land_collapse="average",
    crop_area_frac="cdl_cropland.init",
    crop_area_frac_patch="",
    crop_par="CDL-US-M_CropParameters.csv",
    crop_par_patch="",
    main_path="/gpfs/group/kaf26/default/dcl5300/test-runs_WBM/current/",
    project_ID="PCHES_crop_yield_UC_climate_param",
    spinup_loops=5,
)

check jobid: 43727759
spool jobid: 43727904
run jobid: 43727905
